LIMPIAR AMBOS CSV

In [4]:
import sys
from pathlib import Path
import pandas as pd

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent.parent/"src"
sys.path.append(str(path_inicio))
from utilidades.constantes import MOSTRADOR_SALIDA_PATH, SERVICIOS_SALIDA_PATH, ARCHIVO_MOSTRADOR_FINAL_PATH, ARCHIVO_SERVICIOS_FINAL_PATH

LIMPIO MOSTRADOR

In [3]:

# 1. Leer archivo consolidado
df = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')

# 2. Eliminar ventas anuladas (si existe la columna "Anulado")
if "Anulado" in df.columns:
    df = df[df["Anulado"].str.lower() == "no"]
else:
    print("No se encontró columna 'Anulado'.")

# 3. Estandarizar columna de fecha
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

# 4. Asegurar que TotalVenta y Cantidad sean numéricos
df['TotalVenta'] = pd.to_numeric(df['TotalVenta'].round(0).astype('Int64') , errors='coerce')
if 'Cantidad' in df.columns:
    df['Cantidad'] = pd.to_numeric(df['Cantidad'], errors='coerce')


columnas_utiles = ['Fecha', 'Producto', 'Cantidad', 'TotalVenta','Cond.Pago','Cliente','CUIT']
df = df[columnas_utiles]

# 6. Guardar CSV limpio
df.to_csv(ARCHIVO_MOSTRADOR_FINAL_PATH, index=False, encoding='utf-8-sig')
print("Archivo limpio guardado en data_final/ventas_limpio.csv")

# Verificá primeras filas
df.head()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50912\1260196933.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50912\1260196933.py:11: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')


Archivo limpio guardado en data_final/ventas_limpio.csv


,Fecha,Producto,Cantidad,TotalVenta,Cond.Pago,Cliente,CUIT
0,2023-03-30 17:13:01,ENVIO A DOMICILIO,1.0,63500,.CREDITO,CASINELLI MARIO OSCAR,20077794655.0
1,2023-03-30 11:14:11,BEBIDA GASEOSA POR 2 1/4,5.0,24450,.CREDITO,LABORATORIO ELEA PHOENIX S A,30544640212.0
2,2023-03-30 09:24:52,BEBIDA GASEOSA POR 2 1/4,2.0,13300,EFECTIVO,AGUAS BONAERENSES S.A.,30707919902.0
3,2023-03-29 16:12:46,ALMUERZO,2600.0,2600,.CREDITO,LABORATORIO ELEA PHOENIX S A,30544640212.0
4,2023-03-29 16:10:34,SERVICIO DE CATERING DEL DIA 5 DESEPTIEMBRE,10500.0,10500,.CREDITO,ROEMMERS S A I C F,30500938125.0


ELIMINO EL EFECTO INFLACIONARIO


In [ ]:
# ---------- parámetros ----------
DESTINO_ANIO = 2025
DESTINO_MES  = 9   # 8 = agosto 2025
INCLUIR_MES_ORIGEN = True  # True: multiplica desde el mismo mes; False: desde el mes siguiente
# --------------------------------

# 1) Leer ventas
# Asegurate de que MOSTRADOR_SALIDA_PATH sea un Path (si ya lo es, esta línea no rompe)
ARCHIVO_MOSTRADOR_FINAL_PATH = Path(ARCHIVO_MOSTRADOR_FINAL_PATH)

df = pd.read_csv(ARCHIVO_MOSTRADOR_FINAL_PATH, parse_dates=['Fecha'])
df['Anio'] = df['Fecha'].dt.year
df['Mes']  = df['Fecha'].dt.month

# 2) Inflación mensual en % (ene/2023 → ago/2025)
inflaciones_pct = [
    # 2023
    6.0, 6.6, 7.7, 8.4, 7.8, 6.0, 6.3, 12.4, 12.7, 8.3, 12.8, 25.5,
    # 2024
    20.6, 13.2, 11.0, 8.8, 4.2, 4.6, 4.0, 4.2, 3.5, 2.7, 2.4, 2.7,
    # 2025 (ene–ago)
    2.2, 2.4, 3.7, 2.8, 1.5, 1.6, 1.9, 1.9, 2.1, 2.1
]
infl = [x/100 for x in inflaciones_pct]

# 3) Tabla de meses (ene/2023 → ago/2025)
anios = [2023]*12 + [2024]*12 + [2025]*10
meses = list(range(1,13)) + list(range(1,13)) + list(range(1,11))
tbl = pd.DataFrame({'Anio': anios, 'Mes': meses, 'inf': infl})

# Índice del destino
idx_dest = tbl.index[(tbl.Anio == DESTINO_ANIO) & (tbl.Mes == DESTINO_MES)][0]

# 4) Coeficiente: prod (1+inf_j) desde j_start hasta destino
coef = []
for i in range(len(tbl)):
    if i > idx_dest:
        coef.append(float('nan'))    # no ajustamos posterior al destino
        continue
    j_start = i if INCLUIR_MES_ORIGEN else i+1
    f = 1.0
    for j in range(j_start, idx_dest+1):
        f *= (1 + tbl.loc[j, 'inf'])
    coef.append(f)

tbl['Coef_Ajuste'] = coef

# 5) Merge y ajuste
df = df.merge(tbl[['Anio','Mes','Coef_Ajuste']], on=['Anio','Mes'], how='left')
df['TotalVenta_Ajustado'] = (df['TotalVenta'] * df['Coef_Ajuste']).round(0).astype('Int64')

# 6) Guardar a archivo NUEVO (misma carpeta, nombre con sufijo)
SALIDA = ARCHIVO_MOSTRADOR_FINAL_PATH.parent / f"ventas_mostrador_final.csv"
df.to_csv(SALIDA, index=False, encoding="utf-8-sig")
print(f"Dataset ajustado a precios de {DESTINO_MES:02d}/{DESTINO_ANIO}: {SALIDA}")


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_50912\473569619.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ARCHIVO_MOSTRADOR_FINAL_PATH, parse_dates=['Fecha'])


Dataset ajustado a precios de 09/2025: ..\..\3_data_final\ventas_mostrador_final.csv


LIMPIO SERVICIOS

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# ---------- Parámetros ----------
DESTINO_ANIO  = 2025
DESTINO_MES   = 9    # 9 = septiembre 2025
INCLUIR_MES_ORIGEN = False  # True => multiplica desde el mismo mes; False => desde el mes siguiente
# --------------------------------

# 1) Leer ventas
df = pd.read_csv(SERVICIOS_SALIDA_PATH, parse_dates=['fecha_emision'])

# 2) Normalizar tipos / signos (si aplica para tu dataset)
df['tipo_comprobante'] = df['tipo_comprobante'].astype(str)
df.loc[df['tipo_comprobante'].isin(['3','8','203']), 'importe_total'] *= -1

# 3) Mes y año base
df['Anio'] = df['fecha_emision'].dt.year
df['Mes']  = df['fecha_emision'].dt.month

# 4) Inflación mensual en %
inflaciones_pct = [
    # 2023
    6.0, 6.6, 7.7, 8.4, 7.8, 6.0, 6.3, 12.4, 12.7, 8.3, 12.8, 25.5,
    # 2024
    20.6, 13.2, 11.0, 8.8, 4.2, 4.6, 4.0, 4.2, 3.5, 2.7, 2.4, 2.7,
    # 2025 (ene–ago)
    2.2, 2.4, 3.7, 2.8, 1.5, 1.6, 1.9, 1.9, 2.1, 2.1
]
infl = [x/100 for x in inflaciones_pct]

# 5) Tabla cronológica (ene/2023 → ago/2025)
anios = [2023]*12 + [2024]*12 + [2025]*10
meses = list(range(1,13)) + list(range(1,13)) + list(range(1,11))
tbl = pd.DataFrame({'Anio': anios, 'Mes': meses, 'inf': infl})

# 6) Índice del destino
idx_dest = tbl.index[(tbl['Anio']==DESTINO_ANIO) & (tbl['Mes']==DESTINO_MES)][0]

# 7) Calcular coeficientes acumulados hasta destino
coef = []
for i in range(len(tbl)):
    if i > idx_dest:
        coef.append(float('nan'))
        continue
    j_start = i if INCLUIR_MES_ORIGEN else i+1
    f = np.prod(1 + tbl.loc[j_start:idx_dest, 'inf'])
    coef.append(round(f, 6))
tbl['Coef_Ajuste'] = coef

# ==============================================================
# 🔍 Emparejar notas de crédito con ventas (por CUIT y monto)
# ==============================================================

ventas = df[~df['tipo_comprobante'].isin(['3','8','203'])].copy()
ncs    = df[df['tipo_comprobante'].isin(['3','8','203'])].copy()

ventas['fecha_emision'] = pd.to_datetime(ventas['fecha_emision'])
ncs['fecha_emision']    = pd.to_datetime(ncs['fecha_emision'])

if 'cuit_receptor' not in df.columns:
    raise ValueError("No existe la columna 'cuit_receptor' en el dataset.")

# Inicializar columnas
ncs['Anio_ajuste'] = ncs['Anio']
ncs['Mes_ajuste']  = ncs['Mes']

ventas = ventas.sort_values(['cuit_receptor','fecha_emision'])
ncs    = ncs.sort_values(['cuit_receptor','fecha_emision'])

for idx, row in ncs.iterrows():
    cuit = row['cuit_receptor']
    monto_nc = abs(row['importe_total'])
    fecha_nc = row['fecha_emision']

    # Ventas del mismo CUIT, anteriores o del mismo día
    subset = ventas[(ventas['cuit_receptor'] == cuit) & (ventas['fecha_emision'] <= fecha_nc)]
    if subset.empty:
        continue

    subset = subset.copy()
    subset['diff'] = (subset['importe_total'] - monto_nc).abs()

    # 1️⃣ Intentar coincidencia exacta de monto
    exact_match = subset[subset['diff'] == 0]
    if not exact_match.empty:
        venta_match = exact_match.sort_values('fecha_emision').iloc[-1]  # la última venta igual
    else:
        # 2️⃣ Si no hay exacta, elegir la más cercana
        venta_match = subset.sort_values(['diff','fecha_emision']).iloc[0]

    # Asignar mes/año de esa venta
    ncs.at[idx, 'Anio_ajuste'] = venta_match['fecha_emision'].year
    ncs.at[idx, 'Mes_ajuste']  = venta_match['fecha_emision'].month

# Unir ventas + NCs
df = pd.concat([ventas, ncs], ignore_index=True)

# ==============================================================
# 🔢 Merge con tabla de inflación y ajuste de montos
# ==============================================================
# Asegurar que todas las filas tengan Anio_ajuste y Mes_ajuste
df['Anio_ajuste'] = df['Anio_ajuste'].fillna(df['Anio'])
df['Mes_ajuste']  = df['Mes_ajuste'].fillna(df['Mes'])

df = df.merge(
    tbl[['Anio','Mes','Coef_Ajuste']],
    left_on=['Anio_ajuste','Mes_ajuste'],
    right_on=['Anio','Mes'],
    how='left',
    suffixes=('_emitido','_real')
)

# Limpiar columnas duplicadas
df = df.rename(columns={
    'Anio_emitido': 'Anio',
    'Mes_emitido': 'Mes'
})
df = df.drop(columns=['Anio_real','Mes_real'], errors='ignore')

# Ajustar todos los montos (ventas y NC)
df['TotalVenta_Ajustado'] = (df['importe_total'] * df['Coef_Ajuste']).round(0).astype('Int64')

# ==============================================================
# 💾 Guardar resultado
# ==============================================================

SERVICIOS_SALIDA_PATH = Path(ARCHIVO_SERVICIOS_FINAL_PATH)
SALIDA = SERVICIOS_SALIDA_PATH.parent / "ventas_servicios_final.csv"
df.to_csv(SALIDA, index=False, encoding='utf-8-sig')

print(f"✅ Dataset ajustado guardado como {SALIDA}")
print(f"   Ajustado a precios de {DESTINO_MES:02d}/{DESTINO_ANIO}")


✅ Dataset ajustado guardado como ..\..\3_data_final\ventas_servicios_final.csv
   Ajustado a precios de 09/2025
